In [1]:
import regex as re
import calendar
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn.preprocessing as preprocessing

### Text preprocessing

In [4]:
isw_data = pd.read_csv("..\clean_data\isw.csv")

In [5]:
tg_messages = pd.read_csv("..\clean_data\preprocessed_tg.csv")

In [6]:
tg_messages.head()

,time,date,message
0,23:40:45,2023-01-25,ще декілька бпла на лінії фронту у запорізькій...
1,23:23:13,2023-01-25,збиваються цілі на півдні про кожну не пишемо
2,23:09:57,2023-01-25,дніпро робота ппо
3,22:58:42,2023-01-25,загрози ту22м3 на даний момент немає тільки бпла
4,22:41:15,2023-01-25,є збиття бпла по півдню


In [7]:
isw_data.head()

,date,title,url,html,main_text
0,24-02-2022,Russia-Ukraine Warning Update: Initial Russian...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","february 24, 3:00 pm est russian president vla..."
1,25-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...",russian forces carried out additional air and ...
2,26-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...",russia has surprisingly failed to gain air sup...
3,27-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","february 27, 4pm est the russian military has ..."
4,28-02-2022,"Russian Offensive Campaign Assessment, Februar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","february 28, 3:30pm est the russian military i..."


In [8]:
# test text vectorization 
isw_text_data = isw_data[["date", "main_text"]]

In [9]:
isw_text_data.head()

,date,main_text
0,24-02-2022,"february 24, 3:00 pm est russian president vla..."
1,25-02-2022,russian forces carried out additional air and ...
2,26-02-2022,russia has surprisingly failed to gain air sup...
3,27-02-2022,"february 27, 4pm est the russian military has ..."
4,28-02-2022,"february 28, 3:30pm est the russian military i..."


In [10]:
isw_text_data["main_text"][isw_text_data["main_text"].isnull()]

273    NaN
304    NaN
311    NaN
Name: main_text, dtype: object

In [11]:
isw_text_data = isw_text_data.fillna('')

### Vectorization

Remove:
1) Stop words.
2) Digits.
3) Non-word and non-whitespace characters.

In [12]:
def to_vector_preprocessing(text, stop_words = []):
    if not stop_words:
        stop_words = stopwords.words("english")
    stemmer = PorterStemmer()
    text_array = word_tokenize(re.sub('[\W\s\d]', ' ', text.lower()))
    processed_text = ' '.join( 
            [
            stemmer.stem(word) for word in text_array
            if (len(word) > 2) and (word not in stop_words) 
            ])
    return processed_text

In [13]:
def tfidf_vectorizer(_corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(_corpus)
    sparse_matrix = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
    return sparse_matrix

In [14]:
langs = ['english', 'russian']
stop_words = ['russian'] + list(map(lambda elem: elem.lower(), calendar.month_name))[1:]
for lang in langs:
    stop_words += stopwords.words(lang)

In [15]:
processed_text = isw_text_data["main_text"].apply(lambda row: to_vector_preprocessing(row, stop_words))

In [16]:
sm = tfidf_vectorizer(processed_text.tolist())

In [17]:
sm['date'] = isw_text_data['date']

In [18]:
def plot_top_by_doc(df, n=5):
    fig, ax = plt.subplots(n, figsize=(6, 30))
    for i in range(n):
        df.iloc[i, :].sort_values(ascending=False)[:10].plot.barh(
            ax=ax[i], 
            cmap="jet", 
            title=f"Doc {i}").invert_yaxis()
    plt.subplots_adjust(hspace=0.4)

In [19]:
sm.head()

,abachev,abandon,abbrevi,abc,abdollahian,abduct,abet,abid,abil,abject,...,дивизион,кедр,коридор,набор,окремі,переселения,программа,підрозділи,рбк,сухопутный
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.057502,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.043905,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
tg_messages.head()

,time,date,message
0,23:40:45,2023-01-25,ще декілька бпла на лінії фронту у запорізькій...
1,23:23:13,2023-01-25,збиваються цілі на півдні про кожну не пишемо
2,23:09:57,2023-01-25,дніпро робота ппо
3,22:58:42,2023-01-25,загрози ту22м3 на даний момент немає тільки бпла
4,22:41:15,2023-01-25,є збиття бпла по півдню


In [21]:
tg_processed = tg_messages['message'].apply(lambda row: to_vector_preprocessing(row))

In [22]:
sm_tg = tfidf_vectorizer(tg_processed.tolist())

In [23]:
sm_tg[['date', 'time']] = tg_messages[['date', 'time']]

In [24]:
sm_tg.head()

,_bot,afp,agm,ah,air,akinci,alexand,amev,anpsq,arm,...,їзд,їзди,їнки,їхала,їхньому,їхня,їхні,їхніх,ґрунтовних,date
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25


In [25]:
sm_tg.sort_values(by='date', ascending=True)

,_bot,afp,agm,ah,air,akinci,alexand,amev,anpsq,arm,...,їзд,їзди,їнки,їхала,їхньому,їхня,їхні,їхніх,ґрунтовних,date
4877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-29
4876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-29
4874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-30
4875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-30
4873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-25


In [26]:
sm.head()

,abachev,abandon,abbrevi,abc,abdollahian,abduct,abet,abid,abil,abject,...,дивизион,кедр,коридор,набор,окремі,переселения,программа,підрозділи,рбк,сухопутный
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.057502,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.043905,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
print("ISW processed shape: ", sm.shape)
print("Telegram processed shape: ", sm_tg.shape)

ISW processed shape:  (336, 9846)
Telegram processed shape:  (4878, 9299)


In [28]:
sm["date"] = pd.to_datetime(sm["date"], format='%d-%m-%Y')
sm_tg["date"] = pd.to_datetime(sm_tg["date"], format='%Y-%m-%d')
sm_tg["time"] = pd.to_datetime(sm_tg["time"], format='%H:%M:%S').dt.round('H')

In [29]:
merged_sm = sm.merge(sm_tg, how='outer', on="date")

In [30]:
merged_sm = merged_sm.fillna(0)

In [31]:
merged_sm = merged_sm.set_index(['date', 'time_y'])

In [32]:
merged_sm = merged_sm.reset_index()

In [33]:
merged_sm.head()

,date,time_y,abachev,abandon,abbrevi,abc,abdollahian,abduct,abet,abid,...,їжа,їзд,їзди,їнки,їхала,їхньому,їхня,їхні,їхніх,ґрунтовних
0,2022-02-24,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-02-25,0,0.0,0.057502,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-02-26,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-02-27,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-02-28,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
merged_sm  = merged_sm.groupby("date").sum()

In [35]:
merged_sm.head()

,abachev,abandon,abbrevi,abc,abdollahian,abduct,abet,abid,abil,abject,...,їжа,їзд,їзди,їнки,їхала,їхньому,їхня,їхні,їхніх,ґрунтовних
date,,,,,,,,,,,,,,,,,,,,,
2022-02-24,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-02-25,0.0,0.057502,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-02-26,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-02-27,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-02-28,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.043905,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# merged_sm['hourDatetimeEpoch'] = (merged_sm['date'] + pd.to_timedelta(merged_sm['time_y'], unit='h')).apply(
#     lambda x: int(x.timestamp())
# )

In [37]:
# merged_sm = merged_sm.drop(['date', 'time_y'], axis=1)

In [38]:
# low = []
# for word in new_sm.columns:
#     if len(word) > 12:
#         low.append(word)
# len(low) / len(new_sm.columns)

In [39]:
# merged_sm.to_csv("..\clean_data\SM_preprocessed.csv")

In [40]:
# n1, n2, temp  = 1, round(min(merged_sm.shape) / 2), 0
# pca_new = PCA(n_components=1)
# pca_new.fit(merged_sm)
# while not 0.94 < sum(pca_new.explained_variance_ratio_) < 0.95:
#     if sum(pca_new.explained_variance_ratio_) < 0.95:
#         temp = n2
#         n1 = round((n1 + n2) / 2)
#         n2 = temp
#     else:
#         n2 = n1
#         temp = round((1 + n2) / 2)
#         n1 = (temp + n2)
#     pca_new = PCA(n_components=n1)
#     pca_new.fit(merged_sm)    
#     print("Variance: ", sum(pca_new.explained_variance_ratio_),  "NC: ", n1)
    

In [41]:
# Variance:  0.9234692943324196 NC:  1237
# Variance:  0.9731110532529056 NC:  1855
# Variance:  0.8841993576078081 NC:  928
# Variance:  0.9389528247135009 NC:  1392

### Dimensions reduction

In [42]:
pca = PCA()
new = pca.fit_transform(merged_sm)

In [43]:
sum(pca.explained_variance_ratio_)

1.0000000000000002

In [44]:
merged_sm_2 = pd.DataFrame(new)

In [45]:
merged_sm_2.head()

,0,1,2,3,4,5,6,7,8,9,...,326,327,328,329,330,331,332,333,334,335
0,-9.833173,-0.137912,1.031178,0.718487,0.339813,-0.576608,0.210737,0.155052,0.155647,0.241830,...,0.001358,0.000750,0.000516,0.007649,0.003805,0.007830,0.009322,-0.011622,-0.011459,4.569509e-15
1,-9.838416,-0.097082,0.940281,0.732808,0.337467,-0.680972,0.280662,0.206603,0.152244,0.240537,...,-0.041275,-0.027541,-0.020633,-0.016605,-0.011875,-0.011291,-0.001310,-0.002363,-0.005760,4.569509e-15
2,-9.716154,-0.095405,0.925539,0.716092,0.347916,-0.679477,0.265854,0.200956,0.175628,0.250943,...,0.028931,0.031071,0.015990,-0.019822,0.032921,0.018238,-0.005472,0.003095,0.003267,4.569509e-15
3,-9.771960,-0.073753,0.961972,0.722320,0.323980,-0.623269,0.203734,0.198434,0.154380,0.282542,...,0.026656,0.006528,-0.022216,0.008744,-0.047228,-0.005516,-0.003683,-0.003462,-0.024757,4.569509e-15
4,-9.754662,-0.118964,0.973426,0.756792,0.368118,-0.612103,0.264052,0.216297,0.186797,0.267992,...,-0.032725,-0.049449,-0.007091,0.003396,0.016696,-0.006650,-0.021289,0.009076,0.004213,4.569509e-15


In [46]:
merged_sm_2.shape

(336, 336)

In [47]:
merged_sm_2['date'] = merged_sm.index

### Weather and alarms data preprocessing 

In [48]:
weather_data = pd.read_csv(r"..\clean_data\all_weather_by_hour_v2.csv")

In [49]:
weather_data

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,...,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,hour_icon,hour_source,hour_stations
0,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,91.5,0.0,NaN,0.0,10.0,Overcast,snow,obs,remote
1,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.2,88.2,0.0,NaN,0.0,10.0,Partially cloudy,fog,obs,remote
2,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,10.0,100.0,NaN,NaN,NaN,10.0,Overcast,cloudy,obs,33177099999
3,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.1,92.0,0.0,NaN,0.0,10.0,Overcast,fog,obs,remote
4,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,93.8,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182707,49.2336,28.4486,"Вінниця, Україна","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote
182708,49.2336,28.4486,"Вінниця, Україна","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,10.0,30.0,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,obs,33562099999
182709,49.2336,28.4486,"Вінниця, Україна","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote
182710,49.2336,28.4486,"Вінниця, Україна","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote


In [50]:
weather_data_copy = weather_data.copy()

In [51]:
weather_data_copy["city_resolvedAddress"]= weather_data_copy["city_resolvedAddress"].str.split(',')
weather_data_copy

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,...,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,hour_icon,hour_source,hour_stations
0,50.7469,25.3263,"[Луцьк, Луцький район, Україна]","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,91.5,0.0,NaN,0.0,10.0,Overcast,snow,obs,remote
1,50.7469,25.3263,"[Луцьк, Луцький район, Україна]","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.2,88.2,0.0,NaN,0.0,10.0,Partially cloudy,fog,obs,remote
2,50.7469,25.3263,"[Луцьк, Луцький район, Україна]","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,10.0,100.0,NaN,NaN,NaN,10.0,Overcast,cloudy,obs,33177099999
3,50.7469,25.3263,"[Луцьк, Луцький район, Україна]","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.1,92.0,0.0,NaN,0.0,10.0,Overcast,fog,obs,remote
4,50.7469,25.3263,"[Луцьк, Луцький район, Україна]","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,93.8,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182707,49.2336,28.4486,"[Вінниця, Україна]","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote
182708,49.2336,28.4486,"[Вінниця, Україна]","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,10.0,30.0,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,obs,33562099999
182709,49.2336,28.4486,"[Вінниця, Україна]","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote
182710,49.2336,28.4486,"[Вінниця, Україна]","Vinnytsia,Ukraine",Europe/Kiev,2.0,2023-01-20,1674165600,11.2,5.7,...,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote


In [52]:
weather_data_copy["city_resolvedAddress"] = pd.DataFrame(weather_data_copy["city_resolvedAddress"].to_list())[0]

In [53]:
weather_data_copy.head()

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,...,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,hour_icon,hour_source,hour_stations
0,50.7469,25.3263,Луцьк,"Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,91.5,0.0,NaN,0.0,10.0,Overcast,snow,obs,remote
1,50.7469,25.3263,Луцьк,"Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.2,88.2,0.0,NaN,0.0,10.0,Partially cloudy,fog,obs,remote
2,50.7469,25.3263,Луцьк,"Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,10.0,100.0,NaN,NaN,NaN,10.0,Overcast,cloudy,obs,33177099999
3,50.7469,25.3263,Луцьк,"Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.1,92.0,0.0,NaN,0.0,10.0,Overcast,fog,obs,remote
4,50.7469,25.3263,Луцьк,"Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,93.8,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote


In [54]:
alarms = pd.read_csv(r"..\clean_data\alarms.csv", sep=";")

In [55]:
alarms.head()

,id,region_id,region_title,region_city,all_region,start,end,clean_end,intersection_alarm_id
0,2,3,Вінниччина,Вінниця,0,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 23:41:53,NaN
1,4,12,Львівщина,Львів,0,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 07:15:28,NaN
2,5,14,Одещина,Одеса,0,2022-02-26 07:16:58,2022-02-26 07:47:03,2022-02-26 07:47:03,NaN
3,6,6,Житомирщина,Житомир,0,2022-02-26 08:05:54,2022-02-26 09:36:36,2022-02-26 09:36:36,NaN
4,7,3,Вінниччина,Вінниця,0,2022-02-26 08:39:39,2022-02-26 10:42:41,2022-02-26 10:42:41,NaN


In [56]:
regions = pd.read_csv(r"..\clean_data\regions.csv")
regions.head()

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5


In [57]:
regions = regions.rename(columns={"center_city_ua": "region_city"})

In [58]:
regions["region_id"]  += 1

In [59]:
col_weather = ["city_resolvedAddress","day_datetimeEpoch","hour_datetimeEpoch", "day_datetime", "day_tempmax",
"day_tempmin", "day_temp", "day_dew", "day_humidity",
"day_precip", "day_precipcover",
"day_solarradiation", "day_solarenergy",
"day_uvindex", "day_moonphase", "hour_datetime",
"hour_temp", "hour_humidity",
"hour_precip", "hour_precipprob",
"hour_windgust", "hour_windspeed", "hour_winddir",
"hour_pressure", "hour_visibility",
"hour_cloudcover", "hour_solarradiation",
"hour_uvindex", "hour_severerisk", "hour_conditions"
]

In [60]:
actual_weather = weather_data_copy[col_weather]

In [61]:
actual_weather = actual_weather.rename(columns={"city_resolvedAddress": "region_city"})

In [62]:
actual_weather["region_city"] = actual_weather["region_city"].replace('Хмельницька область', "Хмельницький")

In [63]:
actual_weather["start_level"] = pd.to_datetime(actual_weather["day_datetime"] + ' ' + actual_weather["hour_datetime"])

In [64]:
# actual_weather = actual_weather.merge(regions[["region_city", "region_id"]], on="region_city")

In [65]:
actual_weather = pd.merge(actual_weather, regions, on="region_city")

In [66]:
actual_weather.shape

(182712, 35)

In [67]:
col_alarms = ["region_id", "region_city", "start_level","end_level"]

In [68]:
alarms["start_level"] = pd.to_datetime(alarms["start"]).dt.floor("H")
alarms["end_level"] = pd.to_datetime(alarms["end"]).dt.ceil("H")

In [69]:
alarms_actual = alarms[col_alarms]

In [70]:
alarms_actual.head()

,region_id,region_city,start_level,end_level
0,3,Вінниця,2022-02-25 22:00:00,2022-02-26 00:00:00
1,12,Львів,2022-02-26 06:00:00,2022-02-26 08:00:00
2,14,Одеса,2022-02-26 07:00:00,2022-02-26 08:00:00
3,6,Житомир,2022-02-26 08:00:00,2022-02-26 10:00:00
4,3,Вінниця,2022-02-26 08:00:00,2022-02-26 11:00:00


In [71]:
events_by_hour = []
events_dict = alarms.to_dict('records')
for event in events_dict:
    for time_delta in pd.date_range(start=event["start_level"], end=event["end_level"], freq='1H'):
        event_new = event.copy()
        event_new["hour_level"] = time_delta
        events_by_hour.append(event_new)

In [72]:
actual_alarms_new = pd.DataFrame.from_dict(events_by_hour)

In [73]:
actual_alarms_new.sample(10)

,id,region_id,region_title,region_city,all_region,start,end,clean_end,intersection_alarm_id,start_level,end_level,hour_level
19961,7307,5,️Донеччина,Донецька обл.,1,2022-04-28 15:50:21,2022-04-28 16:03:14,2022-04-28 16:03:14,NaN,2022-04-28 15:00:00,2022-04-28 17:00:00,2022-04-28 17:00:00
17767,5806,2,Волинь,Волинська обл.,1,2022-04-19 01:44:42,2022-04-19 02:26:39,2022-04-19 02:26:39,NaN,2022-04-19 01:00:00,2022-04-19 03:00:00,2022-04-19 03:00:00
31921,19602,10,Кіровоградщина,Кіровоградська обл.,1,2022-06-27 11:30:30,2022-06-27 12:01:19,2022-06-27 12:01:19,NaN,2022-06-27 11:00:00,2022-06-27 13:00:00,2022-06-27 13:00:00
58460,47309,10,Кіровоградщина,Світловодськ,0,2023-01-15 14:22:46,2023-01-15 14:52:37,2023-01-15 14:52:37,NaN,2023-01-15 14:00:00,2023-01-15 15:00:00,2023-01-15 14:00:00
47821,37047,13,Миколаївщина,Миколаївська обл.,1,2022-10-15 06:51:02,2022-10-15 08:15:26,2022-10-15 08:15:26,NaN,2022-10-15 06:00:00,2022-10-15 09:00:00,2022-10-15 09:00:00
47680,36767,23,Чернігівщина,Чернігівська обл.,1,2022-10-13 18:54:11,2022-10-13 19:03:55,2022-10-13 19:03:55,NaN,2022-10-13 18:00:00,2022-10-13 20:00:00,2022-10-13 20:00:00
34586,22333,2,Волинь,Волинська обл.,1,2022-07-18 17:42:41,2022-07-18 18:04:23,2022-07-18 18:04:23,NaN,2022-07-18 17:00:00,2022-07-18 19:00:00,2022-07-18 17:00:00
13874,3853,13,Миколаївщина,Миколаївська обл.,1,2022-04-05 10:02:47,2022-04-05 10:25:23,2022-04-05 10:25:23,NaN,2022-04-05 10:00:00,2022-04-05 11:00:00,2022-04-05 10:00:00
26971,14485,15,Полтавщина,Полтавська обл.,1,2022-05-30 02:57:19,2022-05-30 06:00:20,2022-05-30 06:00:20,NaN,2022-05-30 02:00:00,2022-05-30 07:00:00,2022-05-30 04:00:00
27887,15487,9,Київщина,Київська обл.,1,2022-06-04 04:11:52,2022-06-04 04:33:40,2022-06-04 04:33:40,NaN,2022-06-04 04:00:00,2022-06-04 05:00:00,2022-06-04 04:00:00


In [74]:
actual_alarms_new["hour_level_datetimeEpoch"] = actual_alarms_new["hour_level"].apply(lambda x: int(x.timestamp()))

In [75]:
actual_weather.head()

,region_city,day_datetimeEpoch,hour_datetimeEpoch,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,...,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,hour_conditions,start_level,region,center_city_en,region_alt,region_id
0,Луцьк,1645653600,1645653600,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,...,91.5,0.0,0.0,10.0,Overcast,2022-02-24 00:00:00,Волинська,Lutsk,Волинь,4
1,Луцьк,1645653600,1645657200,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,...,88.2,0.0,0.0,10.0,Partially cloudy,2022-02-24 01:00:00,Волинська,Lutsk,Волинь,4
2,Луцьк,1645653600,1645660800,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,...,100.0,NaN,NaN,10.0,Overcast,2022-02-24 02:00:00,Волинська,Lutsk,Волинь,4
3,Луцьк,1645653600,1645664400,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,...,92.0,0.0,0.0,10.0,Overcast,2022-02-24 03:00:00,Волинська,Lutsk,Волинь,4
4,Луцьк,1645653600,1645668000,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,...,93.8,0.0,0.0,10.0,Overcast,2022-02-24 04:00:00,Волинська,Lutsk,Волинь,4


### Merge datasets

In [76]:
merged_weather_alarms = actual_weather.merge(actual_alarms_new, how="left", 
                                             left_on=["region_alt", "hour_datetimeEpoch"], 
                                             right_on=["region_title", "hour_level_datetimeEpoch"])

In [77]:
merged_weather_alarms["is_alarm"] = merged_weather_alarms["start"].apply(lambda x: 0 if pd.isnull(x) else 1)

In [78]:
cond_change = preprocessing.LabelEncoder()
merged_weather_alarms["hour_conditions_id"] = cond_change.fit_transform(merged_weather_alarms["hour_conditions"])

In [79]:
merged_weather_alarms.sample(5)

,region_city_x,day_datetimeEpoch,hour_datetimeEpoch,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,...,start,end,clean_end,intersection_alarm_id,start_level_y,end_level,hour_level,hour_level_datetimeEpoch,is_alarm,hour_conditions_id
119314,Запоріжжя,1667944800,1667948400,2022-11-09,7.0,-0.5,2.9,1.7,91.9,0.000,...,2022-11-08 22:33:42,2022-11-08 22:43:04,2022-11-08 22:43:04,NaN,2022-11-08 22:00:00,2022-11-08 23:00:00,2022-11-08 23:00:00,1.667948e+09,1,4
10462,Кропивницький,1653598800,1653627600,2022-05-27,23.1,10.6,17.8,8.6,56.9,0.000,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,0,4
96288,Хмельницький,1672696800,1672754400,2023-01-03,9.7,3.7,6.6,4.0,83.9,0.000,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,0,3
146222,Полтава,1646776800,1646784000,2022-03-09,-1.9,-5.3,-3.7,-6.7,80.3,0.200,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,0,4
106396,Ужгород,1651870800,1651899600,2022-05-07,19.8,9.6,14.4,10.3,78.2,3.131,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,0,4


In [80]:
fields_to_exclude = [
    "id",
    "region_id_x",
    "region_city_y",
    "region_id_y",
    "day_datetimeEpoch",
    "hour_datetime",
#     "hour_datetimeEpoch",
    "region",
    "center_city_en",
    "region_title",
    "clean_end",
    "start_level_x",
    "start_level_y",
    "start",
    "end",
    "hour_level",
    "hour_level_datetimeEpoch",
    "region_alt",
    "region_title",
    "region",
    "all_region",
    "end_level",
    "intersection_alarm_id",
    "hour_conditions"
]

In [81]:
merged_weather_alarms = merged_weather_alarms.drop(fields_to_exclude, axis=1)

In [82]:
merged_weather_alarms.head()

,region_city_x,hour_datetimeEpoch,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,is_alarm,hour_conditions_id
0,Луцьк,1645653600,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,4.17,...,15.5,275.6,1020.0,0.0,91.5,0.0,0.0,10.0,0,3
1,Луцьк,1645657200,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,4.17,...,14.8,280.3,1021.0,0.2,88.2,0.0,0.0,10.0,0,4
2,Луцьк,1645660800,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,4.17,...,14.4,310.0,1022.0,10.0,100.0,NaN,NaN,10.0,0,3
3,Луцьк,1645664400,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,4.17,...,13.3,295.1,1021.0,0.1,92.0,0.0,0.0,10.0,0,3
4,Луцьк,1645668000,2022-02-24,4.9,0.7,2.6,0.0,83.7,0.118,4.17,...,13.3,305.8,1021.0,0.0,93.8,0.0,0.0,10.0,0,3


In [83]:
merged_sm_2.head()

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,date
0,-9.833173,-0.137912,1.031178,0.718487,0.339813,-0.576608,0.210737,0.155052,0.155647,0.241830,...,0.000750,0.000516,0.007649,0.003805,0.007830,0.009322,-0.011622,-0.011459,4.569509e-15,2022-02-24
1,-9.838416,-0.097082,0.940281,0.732808,0.337467,-0.680972,0.280662,0.206603,0.152244,0.240537,...,-0.027541,-0.020633,-0.016605,-0.011875,-0.011291,-0.001310,-0.002363,-0.005760,4.569509e-15,2022-02-25
2,-9.716154,-0.095405,0.925539,0.716092,0.347916,-0.679477,0.265854,0.200956,0.175628,0.250943,...,0.031071,0.015990,-0.019822,0.032921,0.018238,-0.005472,0.003095,0.003267,4.569509e-15,2022-02-26
3,-9.771960,-0.073753,0.961972,0.722320,0.323980,-0.623269,0.203734,0.198434,0.154380,0.282542,...,0.006528,-0.022216,0.008744,-0.047228,-0.005516,-0.003683,-0.003462,-0.024757,4.569509e-15,2022-02-27
4,-9.754662,-0.118964,0.973426,0.756792,0.368118,-0.612103,0.264052,0.216297,0.186797,0.267992,...,-0.049449,-0.007091,0.003396,0.016696,-0.006650,-0.021289,0.009076,0.004213,4.569509e-15,2022-02-28


In [84]:
# sm["date"] = date

In [85]:
merged_weather_alarms["day_datetime"] = pd.to_datetime(merged_weather_alarms["day_datetime"])

In [86]:
# mwa_copy = merged_weather_alarms.copy()

In [87]:
# mwa_copy["date"] = mwa_copy["day_datetime"]

In [88]:
# merged_sm_2["date"] = pd.to_datetime(merged_sm_2["date"])

In [89]:
print(merged_sm_2["date"].dtype, merged_weather_alarms["day_datetime"].dtype)

datetime64[ns] datetime64[ns]


In [90]:
print("WA shape:", mwa_copy.shape, "SM shape:", merged_sm_2.shape)

WA shape: (195084, 30) SM shape: (336, 337)


In [91]:
# final_data = merged_weather_alarms.merge(merged_sm_2, left_on="day_datetime", right_on="date")
# final_data.to_csv(r"..\clean_data\final_data.csv")